<a href="https://colab.research.google.com/github/harrymyers/Chatbot-Projects/blob/main/HarryMyers_LangchainSummarizer_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_openai langchain_core langchain_community langchain faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.3 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
key = userdata.get('open_ai_key')
import os
os.environ["OPENAI_API_KEY"] = key

RAG program that can retrieva data from any website provided its link.

Built with the help of this youtube series made by a fellow South African:

https://www.youtube.com/watch?v=ekpnVh-l3YA&list=PL4HikwTaYE0GEs7lvlYJQcvKhq0QZGRVn

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import create_retrieval_chain

# Retrieve Data
def get_docs():
    loader = WebBaseLoader('https://en.wikipedia.org/wiki/Roswell_That_Ends_Well')
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=20
    )

    splitDocs = text_splitter.split_documents(docs)

    return splitDocs

def create_vector_store(docs):
    embedding = OpenAIEmbeddings()
    vectorStore = FAISS.from_documents(docs, embedding=embedding)
    return vectorStore


def create_chain(vectorStore):
    model = ChatOpenAI(
        temperature=0.4,
        model='gpt-3.5-turbo-1106'
    )

    prompt = ChatPromptTemplate.from_template("""
    Answer the user's question.
    Context: {context}
    Question: {input}
    """)

    # chain = prompt | model
    document_chain = create_stuff_documents_chain(
        llm=model,
        prompt=prompt
    )

    retriever = vectorStore.as_retriever()

    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    return retrieval_chain


docs = get_docs()
vectorStore = create_vector_store(docs)
chain = create_chain(vectorStore)

response = chain.invoke({
    "input": "Summarize the plot of the Futurama episode 'Roswell That Ends Well'.",
})

def format_response(text, max_length=100):
    words = text.split()
    formatted_response = ""
    line_length = 0

    for word in words:
        # Check if adding the next word would exceed the max_length
        if line_length + len(word) + 1 > max_length:
            formatted_response += '\n'
            line_length = 0

        # Add the word to the formatted response
        if line_length > 0:
            formatted_response += ' '
            line_length += 1

        formatted_response += word
        line_length += len(word)

    return formatted_response


print(format_response(response["answer"]) + '\n')
print(response["context"])

In the episode "Roswell That Ends Well," the Futurama crew accidentally travels back in time to 1947
and crash-lands in Roswell, New Mexico. They must navigate the unfamiliar time period and prevent
any major changes to history while trying to repair their ship and return to the future. Throughout
the episode, they encounter various historical figures and events, leading to comedic and
adventurous situations.

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Roswell_That_Ends_Well', 'title': 'Roswell That Ends Well - Wikipedia', 'language': 'en'}, page_content='"Roswell That Ends Well" is the nineteenth episode in the  third season of the American animated television series Futurama, and the 51st episode of the series overall. It originally aired on the Fox'), Document(metadata={'source': 'https://en.wikipedia.org/wiki/Roswell_That_Ends_Well', 'title': 'Roswell That Ends Well - Wikipedia', 'language': 'en'}, page_content='^ Groening, Matt (2003). Futurama season 3 DVD comme